In [1]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

import numpy as np

Using TensorFlow backend.


In [2]:
##Data Aquisition
me_test = np.load('LO_3_0.01/NJ_LO_3_0.01_500000.npy', allow_pickle=True) #Matrix elements - test
momentum_test_r = np.load('LO_3_0.01/PS3_0.01_500000.npy', allow_pickle=True, encoding='bytes')#4-momenta of inputs - test

me_train = np.load('LO_3_0.01/NJ_LO_3_0.01_3000000.npy', allow_pickle=True) #Matrix elements - train
momentum_train_r = np.load('LO_3_0.01/PS3_0.01_3000000.npy', allow_pickle=True, encoding='bytes') #4-momenta of input - train

In [3]:
##Reformat Momentum
momentum_train = np.array([np.ndarray.flatten(np.array(element)[3:, 2:]) for element in momentum_train_r])
momentum_test = np.array([np.ndarray.flatten(np.array(element)[3:, 2:]) for element in momentum_test_r])
##Rescale
scaler = StandardScaler()

momentum_train = scaler.fit_transform(momentum_train) #Rescale on training set
momentum_test = scaler.transform(momentum_test) #Rescale on test set

In [4]:
# Random Model from Tensorflow Website
def build_model(dropout, hidden_layers, optimizer='RMSprop'):
    
  model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=[4]), #could try swish??
    keras.layers.BatchNormalization(),  
    keras.layers.Dropout(dropout),
      
    *np.repeat([keras.layers.Dense(10, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout)], hidden_layers),
    
    keras.layers.Dense(10, activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(dropout),
    
    keras.layers.Dense(1) 
  ])

  model.compile(loss='mse', #gans?
                optimizer=optimizer,
                metrics=['mape'])
  return model

In [5]:
model = KerasRegressor(build_fn=build_model, epochs=1)

dropout = [0.1, 0.2, 0.3]
hidden_layers = [5, 7, 9]
param_grid = dict(dropout=dropout, hidden_layers=hidden_layers)

grid = RandomizedSearchCV(model, param_grid, cv=3, scoring='neg_mean_absolute_error')
grid_result = grid.fit(momentum_train[:10000], me_train[:10000])

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/1
6666/6666 [==============================]6666/6666 [==============================] - 1s 173us/step - loss: 0.7072 - mean_absolute_percentage_error: 3063.1633

Epoch 1/1
6667/6667 [==============================]6667/6667 [==============================] - 1s 185us/step - loss: 0.7764 - mean_absolute_percentage_error: 3703.4688

Epoch 1/1
6667/6667 [==============================]6667/6667 [==============================] - 1s 206us/step - loss: 0.7780 - mean_absolute_percentage_error: 4100.6973

Epoch 1/1
6666/6666 [==============================]6666/6666 [==============================] - 2s 271us/step - loss: 0.7487 - mean_absolute_percentage_error: 3155.5002

Epoch 1/1
6667/6667 [==============================]6667/6667 [==============================] - 2s 291us/step - loss: 1.1107 - mean_absolute_percentage_error: 4882.4546

Epoch 1/1
6667/6667 [==============================]6667/6667 [=========

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/1
10000/10000 [==============================]10000/10000 [==============================] - 5s 550us/step - loss: 1.1750 - mean_absolute_percentage_error: 4676.6107



In [6]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -20208.839950 using {'hidden_layers': 5, 'dropout': 0.2}
-73285.484666 (74877.478866) with: {'hidden_layers': 5, 'dropout': 0.1}
-23359462.567204 (6927336.230272) with: {'hidden_layers': 7, 'dropout': 0.1}
-15782655229.207970 (17110364084.415371) with: {'hidden_layers': 9, 'dropout': 0.1}
-20208.839950 (20182.706280) with: {'hidden_layers': 5, 'dropout': 0.2}
-603842.861838 (853978.737358) with: {'hidden_layers': 7, 'dropout': 0.2}
-30740488332.326706 (43467095280.395302) with: {'hidden_layers': 9, 'dropout': 0.2}
-37321.014077 (37150.902745) with: {'hidden_layers': 5, 'dropout': 0.3}
-38113436.334196 (29336828.590443) with: {'hidden_layers': 7, 'dropout': 0.3}
-16655490361.421835 (12664597841.219374) with: {'hidden_layers': 9, 'dropout': 0.3}


In [8]:
model = build_model(grid_result.best_params_['dropout'],grid_result.best_params_['hidden_layers'])
model.fit(momentum_train, me_train, epochs=1)

Epoch 1/1
3000000/3000000 [==============================]3000000/3000000 [==============================] - 411s 137us/step - loss: 0.5553 - mean_absolute_percentage_error: 893.5349



In [10]:
model.evaluate(momentum_test, me_test, batch_size=32, verbose=1)

500000/500000 [==============================]500000/500000 [==============================] - 22s 43us/step



[161293591609.60614, 3173938184.183808]